<a href="https://colab.research.google.com/github/Hanoufa123/colab-notebooks/blob/main/Arabic_Sentiment_Analysis_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  تحليل آراء المشاركات باستخدام نموذج CAMeL-BERT لتحليل المشاعر من Hugging Face


##  ما هو تحليل المشاعر Sentiment Analysis؟
تحليل المشاعر هو عملية استخدام الذكاء الاصطناعي لاكتشاف المشاعر (مثل إيجابي، سلبي، محايد) في نص معين. يمكن استخدامه لفهم آراء الناس حول منتج أو خدمة أو حتى دورة تدريبية.

---

## 🤖 ما هو نموذج CAMeL-BERT؟
نموذج [CAMeL-Lab/bert-base-arabic-camelbert-mix-sentiment](https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-mix-sentiment) هو نموذج متقدم مدرّب على اللغة العربية لتحليل المشاعر، من إنتاج مختبر CAMeL-Lab في جامعة نيويورك أبوظبي. يعتمد على معمارية BERT الشهيرة، ومدرب على بيانات متنوعة تشمل اللهجات العربية والفصحى.



#  تثبيت المكتبات المطلوبة -1

In [ ]:

!pip install transformers --quiet  # مكتبة النماذج من Hugging Face
!pip install arabic_reshaper python-bidi  # لدعم عرض النصوص العربية في الرسوم البيانية


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 16.2 MB/s eta 0:00:00


# 2- استيراد المكتبات

In [ ]:

from transformers import pipeline  # لتحميل واستخدام نماذج تحليل المشاعر
import pandas as pd  # لمعالجة البيانات
import os  # للتعامل مع الملفات
from google.colab import drive  # لربط Google Drive

#3- الربط مع قوفل درايف لتخزين نتيجة التقييم
ملاحظة:  لاتقومي بتشغيل  الخلية 3 و 4 اذا كان لايمكنك الربط مع google drive

In [ ]:
#  ربط Google Drive
drive.mount('/content/drive')

ValueError: mount failed

# 4- اعداد المسارات
تأكدي من عمل اضافة اختصار لمسار مجلد دورة الكاء الاصطناعي في ملفات الدرايف الخاص بك

لتنفيذ ذلك افتحي الرابط التالي
1-(https://drive.google.com/drive/folders/1Z7HuHDxuEjDirWajt-uFCAtxurNDCn4E?usp=drive_link)

2- بعد فتح المجلد، انقر بزر الماوس الأيمن على مجلد "AI-Course".

3- اختر "إضافة اختصار إلى Drive".

4-اختر مكان وضع الاختصار  ("My Drive").

5-انقر على "إضافة اختصار".


In [ ]:

folder_path = "/content/drive/MyDrive/AI-Course"
csv_path = os.path.join(folder_path, "opinions.csv")

# 5- استخدام النموذج

In [ ]:
# تحميل نموذج تحليل المشاعر من Hugging Face
model = pipeline("sentiment-analysis", model="CAMeL-Lab/bert-base-arabic-camelbert-mix-sentiment")

#  إدخال رأي المشتركة
name = input("raghad ,renad,jomana,wareef")# TO DO    #Hint use input
text =input("")  #Hint use input

#  تحليل الرأي
result = model(text)[0]
label = result["label"]
score = round(result["score"] * 100, 2)

# ترجمة التصنيف
if label == "negative":
    label_ar = "سلبية"
elif label == "positive":
    label_ar = "إيجابية"
else:
    label_ar = "محايدة"

print(f"الرأي: {label_ar}, النسبة: {score}%")


# 6- تخزين النتيجة في ملف مشترك

In [ ]:

#  إعداد البيانات لتخزينها
new_row = {"الاسماء": name, "الاراء": text, "النتيجة": label_ar, "النسبة": score}

#  إضافة البيانات إلى CSV أو إنشاء الملف إن لم يكن موجودًا

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
else:
    df = pd.DataFrame(columns=["الاسماء", "الاراء", "النتيجة", "النسبة"])
df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
df.to_csv(csv_path, index=False)

print(" تم حفظ رأيك بنجاح في الملف المشترك.")

# 7- تصوير النتائج

In [ ]:
#  عرض الرسوم البيانية لنتائج الآراء

import matplotlib.pyplot as plt
import arabic_reshaper
from bidi.algorithm import get_display

# قراءة البيانات
df = pd.read_csv(csv_path)

# إزالة التكرار حسب الاسم
df_unique = df.drop_duplicates(subset=['الاسماء'])

# حساب التكرار
counts = df_unique['النتيجة'].value_counts()
labels = [get_display(arabic_reshaper.reshape(label)) for label in counts.index]

# تحديد الألوان
color_map = {
    get_display(arabic_reshaper.reshape("سلبية")): 'red',
    get_display(arabic_reshaper.reshape("إيجابية")): 'green',
    get_display(arabic_reshaper.reshape("محايدة")): 'yellow'
}
bar_colors = [color_map[label] for label in labels]

# رسم الأعمدة والدائرة
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# --- رسم الأعمدة ---
bars = axes[0].bar(labels, counts.values, color=bar_colors)
axes[0].set_title(get_display(arabic_reshaper.reshape("نتائج تحليل آراء الطالبات")))
axes[0].set_xlabel(get_display(arabic_reshaper.reshape("الرأي")))
axes[0].set_ylabel(get_display(arabic_reshaper.reshape("عدد التقييمات")))
axes[0].grid(False)

# كتابة العدد فوق الأعمدة
for bar in bars:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2.0, height, f'{int(height)}', ha='center', va='bottom', fontsize=12)

# --- رسم الدائرة ---
pie_colors = [color_map[label] for label in labels]
axes[1].pie(counts.values, labels=labels, autopct='%1.1f%%', colors=pie_colors)
axes[1].set_title(get_display(arabic_reshaper.reshape("النسبة المئوية للآراء")))

plt.tight_layout()
plt.show()

made by : Salwa & Hanouf